In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoModel, AutoTokenizer
import nltk
import json
import pandas as pd
from datasets import *
import numpy as np
import torch

nltk.download('punkt')
SEED = 42

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Luka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
rouge = load_metric("rouge")
model_path = "../../models"
data_path_pairs = '../../data/processed/prompt_reply_pairs.csv'
# data_path_conversations = '../../data/processed/context_reply_pairs.csv'

C:\Users\Luka\AppData\Local\Temp\ipykernel_3600\1601211067.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


In [9]:
data = pd.read_csv(data_path_pairs, sep=";")
data

,prompt,reply,rank,split
0,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...",Kaj pa igranje lažnivca? Za to igro so potrebn...,0,train
1,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...","Uno: Je klasična igra za otroke in odrasle, pr...",1,train
2,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...",Igrate lahko:\n - 8 noro\n - celina\n - Lažnivec,2,train
3,Napišite hvalnico šolskemu taboru v slogu Mela...,Spomnim se vas prijatelji\nIn spet sva se sreč...,0,train
4,Napišite hvalnico šolskemu taboru v slogu Mela...,"Najljubši kamp, ​​krila za naše sanje!\nDaješ ...",1,train
...,...,...,...,...
26173,Kdo je bil Francisco Morazan?,Francisco Morazán je bil honduraški vojaški in...,0,test
26174,Kdo je bil Francisco Morazan?,Francisco Morazán je bil honduraški vojak in p...,1,test
26175,"Imam elektronski set bobnov, a vsakič, ko igra...","Obstaja več stvari, s katerimi lahko poskusite...",0,test
26176,"Imam elektronski set bobnov, a vsakič, ko igra...",Eden od načinov za zmanjšanje tresljajev vašeg...,1,test


In [16]:
test_data = Dataset.from_pandas(data[data['split'] == 'test'])
test_data

Dataset({
    features: ['prompt', 'reply', 'rank', 'split', '__index_level_0__'],
    num_rows: 2610
})

In [17]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [19]:
model_name = "prefix-in-t5-sl-small-finetuned-assistant"
tokenizer = AutoTokenizer.from_pretrained(f"{model_path}/{model_name}")
model = AutoModelForSeq2SeqLM.from_pretrained(f"{model_path}/{model_name}")


# model.predict(test_dataset=test_data)

In [23]:
model.

T5ForConditionalGeneration(
  (shared): Embedding(32000, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32000, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 